In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [1]:
from pydantic import BaseModel
from typing import List
# A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.
# Make classes based on the output description
class SubSection(BaseModel):
    title:str
    content:str

class BlogPost(BaseModel):
    title:str
    introduction:str
    sections: List[SubSection]
    sources: List[str]
    hashtags: List[str]
    
    # By doing this, you can approach the data by result.sections, result.sections[i].title or result.sections[i].content etc.

In [2]:
from crewai import Crew, Agent, Task
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [3]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [ ]:
researcher = Agent(
    role="Senior Researcher",
    goal="Search the web, extract and analyze information",
    backstory="""
    You produce the highest quality research possible.
    You use multiple sources of information and you always double check your sources to make sure they are true and up-to-date.
    You want to impress your coworkers with your work.
    """,
    allow_delegation=False,
    verbose=True,
    # max_iter/ max_execution_time can be used to limit the number of tokens used for the task.
    tools=[
        search_tool,
        scrape_tool,
    ]
)

editor = Agent(
    role="Senior Writer/Editor",
    goal="Write engaging blog posts",
    backstory="""
    You write content that keeps people engaged and entertained.
    Your content is easy to read and it is informative and it makes people want to share it with their friends.
    You are working for a very important client.
    """,
    verbose=True,
)

task = Task(
    description="Write a blog post about {topic}",
    agent=editor,
    expected_output="A blog post with an introduction, at least three sub-sections of content, links to sources, a set of suggested hashtags for social media and a catchy title.",
    output_file="blog_post.md",
    output_pydantic=BlogPost, # or output_json=BlogPost to get JSON results
)

crew = Crew(agents=[researcher, editor,], tasks=[task], verbose=2)

result = crew.kickoff(
    inputs=dict(
        topic="The biggest box office flops of 2024",
    )
)